# Experiments related to General Modbus Reader

In [168]:
import time
import struct
import logging
from pymodbus.client.sync import ModbusTcpClient as ModbusClient
import pymodbus


In [50]:
host = 'EM56akvill.dyndns.org'
port = 30000
with ModbusClient(host=host, port=port) as client:
    #for r in range(2080, 2090):
    r = 2080
    result = client.read_input_registers(r, 10, unit=1)
    print(result.registers)

[34, 0, 0, 0, 206, 0, 19730, 59, 1, 1]


In [172]:
import base_reader
settings = base_reader.DummySettings()

device1 = ('abc.dyndns.org', 30000, {'endian': 'little'})
device1_sensors = (
    (2084, 'heat_rate'),
    (2086, 'total_heat', {'datatype': 'uint32'}),
    (2105, 'ret_temp', {'transform': 'val/10'}),
    (2104, 'sup_temp', {'transform': 'val/10'}),
)

device2 = ('abc.dyndns.org', 30000, {'endian': 'little'})
device2_sensors = (
    (2103, 'hsout', {'transform': 'val/10'}),
    (2102, 'hsin', {'transform': 'val/10'}),
)

device3 = ('def.dyndns.org', 37770, {'endian': 'little'})
device3_sensors = (
    (30, 'heat_rate', {'datatype': 'float32'}),
    (6, 'flow', {'datatype': 'float32', 'transform': 'val/60'}),
)

settings.MODBUS_TARGETS = (
    (device1, device1_sensors),
    (device2, device2_sensors),
    (device3, device3_sensors)
)

class ModbusTCPreader(base_reader.Reader):

    def read(self):

        # list to hold final readings
        readings = []

        for device_info, sensors in self._settings.MODBUS_TARGETS:

            # use the same timestamp for all of the sensors on this device
            ts = time.time()
            try:
                try:
                    host, port, kwargs = device_info
                except:
                    host, port = device_info
                    kwargs = {}
                device_addr = kwargs.get('device_addr', 1)
                endian = kwargs.get('endian', 'big')

                if endian not in ('big', 'little'):
                    raise ValueError(f'Improper endian value for Modbus device {device_info}') 

                print('\n', host, port, device_addr, endian)
                with ModbusClient(host=host, port=port) as client:
                    for sensor_info in sensors:
                        try:
                            try:
                                register, sensor_name, kwargs = sensor_info
                            except:
                                register, sensor_name = sensor_info
                                kwargs = {}
                            
                            datatype = kwargs.get('datatype', 'uint16')
                            transform = kwargs.get('transform', None)
                            register_type = kwargs.get('register_type', 'holding')
                            reading_type = kwargs.get('reading_type', 'value')

                            # determine number of registers to read and the correct struct
                            # unpacking code based upon the data type for this sensor.
                            try:
                                reg_count, unpack_fmt = {
                                    'uint16': (1, 'H'),
                                    'int16': (1, 'h'),
                                    'uint32': (2, 'I'),
                                    'int32': (2, 'i'),
                                    'float': (2, 'f'),
                                    'float32': (2, 'f'),
                                    'double': (4, 'd'),
                                    'float64': (4, 'd'),
                                }[datatype]
                            except:
                                logging.exception(f'Invalid Modbus Datatype: {datatype} for Sensor {sensor_info}')
                                continue

                            # Determine the correct function to use for reading the values
                            try:
                                read_func = {
                                    'holding': client.read_holding_registers,
                                    'input': client.read_input_registers,
                                    'coil': client.read_coils,
                                    'discrete': client.read_discrete_inputs
                                    }[register_type]
                            except:
                                logging.exception(f'Invalid Modbus register type for Sensor {sensor_info}')
                                continue

                            try:
                                reading_type_code = {
                                    'value': base_reader.VALUE,
                                    'state': base_reader.STATE,
                                    'counter': base_reader.COUNTER
                                }[reading_type]
                            except:
                                logging.exception(f'Invalid Reading Type for Sensor {sensor_info}')
                                continue


                            result = read_func(register, reg_count, unit=device_addr)
                            if not hasattr(result, 'registers'):
                                raise ValueError(f'An error occurred while reading Sensor {sensor_info} from Modbus Device {device_info}')
                            
                            # make an array of register values with least-signifcant value first
                            registers = result.registers

                            # calculate the integer equivalent of the registers read
                            if endian == 'big':
                                registers = reversed(registers)
                            val = 0
                            mult = 1
                            for reg in registers:
                                val += reg * mult
                                mult *= 2**16

                            # Use the struct module to convert this number into the appropriate data type.
                            # First, create a byte array that encodes this unsigned number according to 
                            # how many words it contains.
                            reg_count_to_pack_fmt = {
                                1: 'H',
                                2: 'I',
                                4: 'Q'
                            }
                            pack_fmt = reg_count_to_pack_fmt[reg_count]
                            packed_bytes = struct.pack(pack_fmt, val)
                            # unpack bytes to convert to correct datatype
                            val = struct.unpack(unpack_fmt, packed_bytes)[0]

                            if transform:
                                val = eval(transform)
                            sensor_id = f'{self._settings.LOGGER_ID}_{sensor_name}'
                            readings.append( (ts, sensor_id, val, reading_type_code) )                                

                        except Exception as err:
                            logging.exception(str(err))
                            continue    # on to next sensor

            except Exception as err:
                logging.exception(str(err))
                continue   # on to next device

        return readings

ModbusTCPreader(settings).read()


 EM56akvill.dyndns.org 30000 1 little

 EM56akvill.dyndns.org 30000 1 little

 em08akvill.dyndns.org 37770 1 little


[(1594361833.275705, 'test_heat_rate', 206, 1),
 (1594361833.275705, 'test_total_heat', 3892238, 1),
 (1594361833.275705, 'test_ret_temp', 154.8, 1),
 (1594361833.275705, 'test_sup_temp', 155.8, 1),
 (1594361833.7202737, 'test_hsout', 179.3, 1),
 (1594361833.7202737, 'test_hsin', 156.4, 1),
 (1594361833.900743, 'test_heat_rate', 5708955.5, 1),
 (1594361833.900743, 'test_flow', 17.688413492838542, 1)]

'test'